<a href="https://colab.research.google.com/github/sbertolin/03MAIR-Algoritmos-de-optimizacion/blob/master/AG2/Santiago_Bertol%C3%ADn_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AG2** - Actividad Guiada 2<br>
Santiago Bertolín Martínez <br>
Url -> https://github.com/sbertolin/03MAIR-Algoritmos-de-optimizacion/tree/master/AG2

In [0]:
import math
import random

In [0]:
from time import time
#Función para calcular el tiempo de ejecución
def calcular_tiempo(f):
   
    def wrapper(*args, **kwargs):        
        inicio = time()       
        resultado = f(*args, **kwargs)       
        tiempo = time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado
    
    return wrapper

In [0]:
# Comprobar tipus
# NO  if type(A) != type(B):

def distancia(A,B):
  if type(A) is int or type(A) is float:
    return abs(B-A)
  else:
    return math.sqrt(sum([ (A[i]-B[i])**2 for i in range(len(A)) ]))

In [0]:
N = 3000
list_2d = [(random.randrange(1, N*10), random.randrange(1, N*10)) for _ in range(N)]

In [0]:
# Fuerza Bruta
#@calcular_tiempo
def brute_force_distance(L):
  best_found = 100000e10
  A,B = (),()
  
  for i in range(len(L)):
    for j in range(i+1, len(L)):
      distance = distancia(L[i], L[j])
      if distance < best_found:
        A,B = L[i],L[j]
        best_found = distance
        
  return (best_found, [A,B])

In [29]:
brute_force_distance(list_2d)

Tiempo de ejecución para algoritmo: 9.71937084197998


(8.94427190999916, ((24772, 384), (24764, 388)))

In [0]:
# Divide y vencerás
def divide_and_conquer(L):
  list_length = len(L)
  if list_length < 10:
    return brute_force_distance(L)
  
  # Dividir en listas grandes
  #pivot = sum([i[0] for i in L]) / list_length
  
  # sorted usa quick_sort
  sorted_list = sorted(L, key=lambda x: x[0])
  low_list = sorted_list[:list_length//2]
  high_list = sorted_list[list_length//2:]
  
  d_low, low_points = divide_and_conquer(low_list)
  d_high, high_points = divide_and_conquer(high_list)
  
  return brute_force_distance(low_points + high_points)

@calcular_tiempo
def test():
  return divide_and_conquer(list_2d)

In [47]:
print(test())

Tiempo de ejecución para algoritmo: 0.036710262298583984
(7.0, [(10357, 24772), (10364, 24772)])


In [0]:
def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta: 
    #print("Ir a :" + str(desde))
    return desde
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) + ',' + str(RUTA[desde][hasta])

In [56]:
# Problema: Paseo por el rio

TARIFFS = [
    [0,  5,  4,  3,  999,999,999],
    [999,0,  999,2,  3,  999,11],
    [999,999,0,  1,  999,4,  10],
    [999,999,999,0,  5,  6,  9],
    [999,999,999,999,0,  999,4],
    [999,999,999,999,999,0,  3],
    [999,999,999,999,999,999,0]]

def prices(tariffs):
  length = len(tariffs[0])
  
  price_arr = [ [9999]*length for i in [9999]*length ]
  route = [ [""]*length for i in [9999]*length ]
  
  for i in range(length-1):
    for j in range(i+1,length-1):
      MIN = tariffs[i][j]
      route[i][j] = i
      
      for k in range(i,j):
        if price_arr[i][k] + tariffs[k][j] < MIN:
          #MIN = min(MIN, high_prices[i][k] + tariffs[k][j])
          MIN = price_arr[i][k] + tariffs[k][j]
          route[i][j] = k
          
      price_arr[i][j] = MIN
      
  return price_arr, route

([[9999, 5, 4, 3, 8, 8, 9999], [9999, 9999, 999, 2, 3, 8, 9999], [9999, 9999, 9999, 1, 6, 4, 9999], [9999, 9999, 9999, 9999, 5, 6, 9999], [9999, 9999, 9999, 9999, 9999, 999, 9999], [9999, 9999, 9999, 9999, 9999, 9999, 9999], [9999, 9999, 9999, 9999, 9999, 9999, 9999]], [['', 0, 0, 0, 1, 2, ''], ['', '', 1, 1, 1, 3, ''], ['', '', '', 2, 3, 2, ''], ['', '', '', '', 3, 3, ''], ['', '', '', '', '', 4, ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', '']])


In [65]:
ret_prices, ret_routes = prices(TARIFFS)

  
print("\nLa ruta es:")
#calcular_ruta(ret_prices, 0,5)
print(ret_prices)
print('')
print(ret_routes)


La ruta es:
[[9999, 5, 4, 3, 8, 8, 9999], [9999, 9999, 999, 2, 3, 8, 9999], [9999, 9999, 9999, 1, 6, 4, 9999], [9999, 9999, 9999, 9999, 5, 6, 9999], [9999, 9999, 9999, 9999, 9999, 999, 9999], [9999, 9999, 9999, 9999, 9999, 9999, 9999], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]

[['', 0, 0, 0, 1, 2, ''], ['', '', 1, 1, 1, 3, ''], ['', '', '', 2, 3, 2, ''], ['', '', '', '', 3, 3, ''], ['', '', '', '', '', 4, ''], ['', '', '', '', '', '', ''], ['', '', '', '', '', '', '']]
